In [ ]:
# Install necessary libraries
!pip install python-whois dnspython beautifulsoup4 requests

import whois
import dns.resolver
import requests
from bs4 import BeautifulSoup
import json


In [ ]:
# ===== Step 1: Set target domain =====
target_domain = "example.com"  # Change this to any domain you own or a public one

print(f"=== Reconnaissance Report for {target_domain} ===\n")



In [ ]:
# ===== Step 2: WHOIS Lookup =====
try:
    domain_info = whois.whois(target_domain)
    print("----- WHOIS Information -----")
    print(f"Registrar: {domain_info.registrar}")
    print(f"Creation Date: {domain_info.creation_date}")
    print(f"Expiration Date: {domain_info.expiration_date}")
    print(f"Name Servers: {domain_info.name_servers}\n")
except Exception as e:
    print(f"WHOIS Lookup failed: {e}")



In [ ]:
# ===== Step 3: DNS Records =====
try:
    print("----- DNS Records -----")
    for record_type in ["A", "MX", "NS", "TXT"]:
        try:
            answers = dns.resolver.resolve(target_domain, record_type)
            print(f"{record_type} Records:")
            for rdata in answers:
                print(f"  {rdata.to_text()}")
        except Exception:
            print(f"  No {record_type} record found.")
    print()
except Exception as e:
    print(f"DNS lookup failed: {e}")

# This loop will go through four DNS record types:

# A → IPv4 address of the domain.

# MX → Mail server records (for email delivery).

# NS → Name servers that handle DNS queries for the domain.

# TXT → Text records, often used for SPF, DKIM, or verification codes.

# The loop makes the code reusable for different record types without repeating commands.

In [ ]:
# ===== Step 4: Public Breach Check (HaveIBeenPwned API - requires key) =====
# Optional: You can use HaveIBeenPwned API with an API key
# Replace API_KEY with your key if you have one from https://haveibeenpwned.com/API/v3

HIBP_API_KEY = ""  # Add key if you have it
email_to_check = "test@example.com"  # Replace with a test email you control

if HIBP_API_KEY:
    url = f"https://haveibeenpwned.com/api/v3/breachedaccount/{email_to_check}"
    headers = {"hibp-api-key": HIBP_API_KEY, "user-agent": "python"}
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        breaches = resp.json()
        print("----- Breach Information -----")
        for breach in breaches:
            print(f"Breach: {breach['Name']} | Date: {breach['BreachDate']}")
    elif resp.status_code == 404:
        print("No breaches found.")
    else:
        print("Error checking breach data:", resp.status_code)



In [ ]:
# ===== Step 5: Basic Website Info (HTML Title & Meta Tags) =====
try:
    print("\n----- Website Metadata -----")
    r = requests.get(f"http://{target_domain}")
    soup = BeautifulSoup(r.text, "html.parser")
    print("Title:", soup.title.string if soup.title else "No title found")
    for meta in soup.find_all("meta"):
        if meta.get("name") or meta.get("property"):
            print(f"{meta.get('name') or meta.get('property')}: {meta.get('content')}")
except Exception as e:
    print(f"Website metadata extraction failed: {e}")